# TABLEAU PROJECT

## Geospatial Business Intelligence (BI)


#### El objetivo del proyecto es determinar el punto geográfico donde recomendariamos abrir otra sede de nuestra empresa, teniendo en cuenta los siguientes datos:

   Debemos realizar un análisis geoespacial del dataset "companies"

   Datos sobre nuestra empresa:

   - Es una empresa de software de 50 empleados.

   - La empresa diseña videojuegos.

   - Puestos en nuestra compañía: 20 desarrolladores, 20 Diseñadores/creativos/UX/UI y 10 ejecutivos/managers

   Criterios para la localización de la nueva sede:

   - Debe haber ingenieros de software alrededor

   - Debe haber un buen ratio de grandes compañías vs startups

   - Asegurate de que las compañias cercanas cubren los intereses de tu equipo

   - Evita las viejas empresas, rodeate de empresas nuevas

        

1. Importamos las librerias necesarias para trabajar:

In [1]:
import pymongo
MongoClient = pymongo.MongoClient
import time
client = MongoClient()
import pandas as pd
import re
db = client.companies

In [2]:
new_companies = db.companies.find({'founded_year': { '$gte': 2008} })
len(list(new_companies))

3308

2. Para seleccionar empresas que se dediquen a actividades similares a la nuestra, buscamos todos los "category_code" de las empresas de la BBDD:

In [3]:
cursor2 = db.companies.find({'category_code':{'$exists': 'true'}})
category_codes = list(map(lambda r:r['category_code'], list(cursor2)))

category_code_clean = []
for i in category_codes:
    if i not in category_code_clean:
        category_code_clean.append(i)
    else:
        pass
print(category_code_clean)

['enterprise', 'web', 'software', 'news', 'network_hosting', 'games_video', 'music', 'social', 'mobile', 'search', 'advertising', 'messaging', 'security', 'photo_video', 'finance', 'ecommerce', 'hardware', 'travel', 'public_relations', 'other', 'real_estate', 'semiconductor', 'analytics', 'health', 'legal', 'sports', 'biotech', 'cleantech', 'education', 'consulting', 'transportation', None, 'hospitality', 'fashion', 'nonprofit', 'nanotech', 'automotive', 'design', 'manufacturing', 'government', 'local', 'medical']


3. Vamos a seleccionar las empresas que se dedican a : 'web', 'software', 'network_hosting', 'games_video', 'ecommerce', 'hardware', 'analytics':

In [6]:
category_companies = db.companies.find({'$and':
                                       [{'founded_year': { '$gte': 2008} }, 
                                         { '$or': 
                                          [{'category_code': 'web'}, 
                                           {'category_code':'software'}, 
                                           {'category_code':'network_hosting'},
                                           {'category_code': 'games_video'},
                                           {'category_code':'ecommerce'},
                                           {'category_code':'hardware'}, 
                                           {'category_code':'network_hosting'},
                                           {'category_code': 'games_video'},
                                           {'category_code':'ecommerce'},
                                           {'category_code':'hardware'}]}]})
len(list(category_companies))

1971

4. Nos quedamos con las empresas que no tengan año de Deadpool:

In [4]:
category_companies_no_dp = db.companies.find({'$and':
                                       [{'founded_year': { '$gte': 2008} }, 
                                         { '$or': 
                                          [{'category_code': 'web'}, 
                                           {'category_code':'software'}, 
                                           {'}category_code':'network_hosting'},
                                           {'category_code': 'games_video'},
                                           {'category_code':'ecommerce'},
                                           {'category_code':'hardware'}, 
                                           {'category_code':'network_hosting'},
                                           {'category_code': 'games_video'},
                                           {'category_code':'ecommerce'},
                                           {'category_code':'hardware'}]},
                                       {'deadpooled_year': { '$type': 10 }}]})
len(list(category_companies_no_dp))

1824

5. Analizamos los datos del campo "total_money_raised":

In [5]:
cursor3 = db.companies.find({'total_money_raised':{'$exists': 'true'}})
total_money_raised = list(map(lambda r:r['total_money_raised'], list(cursor3)))

total_money_raised_list = []
for i in total_money_raised:
    if i not in total_money_raised_list:
        total_money_raised_list.append(i)
    else:
        pass
print(max(total_money_raised_list))
print(min(total_money_raised_list))

€9M
$0


6. Eliminamos las que total_money_raised = '$0':

In [6]:
category_companies_no_dp_act = db.companies.find({'$and':
                                       [{'founded_year': { '$gte': 2008} }, 
                                         { '$or': 
                                          [{'category_code': 'web'}, 
                                           {'category_code':'software'}, 
                                           {'category_code':'network_hosting'},
                                           {'category_code': 'games_video'},
                                           {'category_code':'ecommerce'},
                                           {'category_code':'hardware'}, 
                                           {'category_code':'network_hosting'},
                                           {'category_code': 'games_video'},
                                           {'category_code':'ecommerce'},
                                           {'category_code':'hardware'}]},
                                       {'deadpooled_year': { '$type': 10 }},
                                       {'total_money_raised':{'$nin': ['$0']}}]})
x = list(category_companies_no_dp_act)
len(x)

245

7. Del listado de las 245 empresas, creamos la Dataframe: 

In [8]:
df = pd.DataFrame(x)
df.head()

,_id,acquisition,acquisitions,alias_list,blog_feed_url,blog_url,category_code,competitions,created_at,crunchbase_url,...,phone_number,products,providerships,relationships,screenshots,tag_list,total_money_raised,twitter_username,updated_at,video_embeds
0,52cdef7c4bab8bd675297e30,None,[],None,,http://goingon.com/blog,software,"[{'competitor': {'name': 'Ning by Glam Media',...",Fri Jul 20 04:55:26 UTC 2007,http://www.crunchbase.com/company/goingon,...,415-200-4606,"[{'name': 'GoingOn', 'permalink': 'goingon'}, ...",[],"[{'is_past': False, 'title': 'CEO', 'person': ...",[],"socialnetworking, enterprise, social-media, so...",$8.5M,GoingOn_Network,Thu Apr 04 09:03:17 UTC 2013,[]
1,52cdef7c4bab8bd675297f76,None,[],,,,games_video,[],Thu Aug 30 11:29:22 UTC 2007,http://www.crunchbase.com/company/crunchyroll,...,,"[{'name': 'crunchyroll.com', 'permalink': 'cru...",[],"[{'is_past': False, 'title': 'beneficial owner...",[],"crunchyroll, anime, manga, video, gaming, musi...",$4.8M,crunchyroll,Fri Dec 20 11:00:02 UTC 2013,[]
2,52cdef7c4bab8bd675297fec,None,[],,http://blog.muktar.com/feed/,http://blog.muktar.com,web,"[{'competitor': {'name': 'Kluster', 'permalink...",Sat Sep 08 08:24:15 UTC 2007,http://www.crunchbase.com/company/fixya,...,,"[{'name': 'Fixya', 'permalink': 'fixya'}]","[{'title': 'Accounting', 'is_past': False, 'pr...","[{'is_past': False, 'title': 'Founder & CEO', ...",[],"fixya, crowdsourcing, diy, repair, customer-se...",$8M,MUaliyu,Tue Nov 19 03:48:00 UTC 2013,"[{'embed_code': '<div style=""width:320px;""><di..."
3,52cdef7c4bab8bd67529801f,None,[],,,http://corp.wamba.com,web,"[{'competitor': {'name': 'Vostu', 'permalink':...",Fri Sep 28 09:37:07 UTC 2007,http://www.crunchbase.com/company/wamba,...,,"[{'name': 'Wamba', 'permalink': 'wamba'}]","[{'title': 'IT Services Provider', 'is_past': ...","[{'is_past': True, 'title': 'Founder', 'person...",[],socialnetwork,€3M,,Wed May 29 18:58:44 UTC 2013,[]
4,52cdef7c4bab8bd67529803d,"{'price_amount': None, 'price_currency_code': ...","[{'price_amount': None, 'price_currency_code':...",,http://www.uptake.com/blog/feed,http://www.uptake.com/blog/,web,"[{'competitor': {'name': 'TripAdvisor', 'perma...",Thu Oct 11 12:07:18 UTC 2007,http://www.crunchbase.com/company/uptake,...,(650) 329-9888,"[{'name': 'Uptake', 'permalink': 'kango'}]","[{'title': '', 'is_past': False, 'provider': {...","[{'is_past': False, 'title': 'VP of Revenue', ...",[],"travel, search, naturallanguage, semantic",$16.8M,uptake,Mon Dec 23 11:36:17 UTC 2013,[]


8. Eliminamos las filas cuya columna Offices no tiene info:

In [9]:
df['l'] = df.offices.apply(len)

9. Creamos dos columnas ('lat' y 'long'), con los datos que estaban incluidos en la columna 'office':

In [12]:
df1 = df.loc[df['l'] > 0]
df1.reset_index(inplace=True)
df1.offices[0]
df1.offices[0][0]['longitude'] # df.offices -> el primer registro, la primera office, latitude

df1['lat'] = df1.apply(lambda row: row['offices'][0]['latitude'], axis=1)
df1['long'] = df1.apply(lambda row: row['offices'][0]['longitude'], axis=1)

/home/laura/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/laura/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


10. Eliminamos las filas que no tienen datos de 'lat' y 'ĺong':

In [14]:
df1['lat'].isnull().sum()

df2 = df1.dropna(subset=['lat'])
df2.reset_index(inplace=True)

11. Unimos latitud y longitud en una columna, que llamaremos "coord":

In [15]:
df2['coord'] = df2[['long', 'lat']].apply(list, axis=1)
df2.coord.head()

/home/laura/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


0    [-122.392142, 37.782263]
1    [-122.393229, 37.781265]
2    [-122.323895, 37.566879]
3    [-122.161674, 37.443058]
4     [-73.985433, 40.740231]
Name: coord, dtype: object

12. Para poder importarlo en MongoDB Compass, y crear el indice Geoespacial que necesitamos, utilizamos la siguiente función:

In [16]:
def toGeoJSON(array):
    return {
       "type":"Point",
       "coordinates":array}
df2['position'] = df2.apply(lambda r:toGeoJSON(r['coord']),axis=1)
df2.position.head()


/home/laura/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


0    {'type': 'Point', 'coordinates': [-122.392142,...
1    {'type': 'Point', 'coordinates': [-122.393229,...
2    {'type': 'Point', 'coordinates': [-122.323895,...
3    {'type': 'Point', 'coordinates': [-122.161674,...
4    {'type': 'Point', 'coordinates': [-73.985433, ...
Name: position, dtype: object

13. De la Dataframe resultante, seleccionamos las columnas que nos interesan:

In [22]:
df3=df2[['name','category_code','founded_year','number_of_employees', 'total_money_raised', 'position']]
df3.head()

,name,category_code,founded_year,number_of_employees,total_money_raised,position
0,GoingOn,software,2008,40.0,$8.5M,"{'type': 'Point', 'coordinates': [-122.392142,..."
1,Crunchyroll,games_video,2008,50.0,$4.8M,"{'type': 'Point', 'coordinates': [-122.393229,..."
2,Fixya,web,2013,30.0,$8M,"{'type': 'Point', 'coordinates': [-122.323895,..."
3,Uptake,web,2008,NaN,$16.8M,"{'type': 'Point', 'coordinates': [-122.161674,..."
4,BeliefNet,web,2008,NaN,$7M,"{'type': 'Point', 'coordinates': [-73.985433, ..."


14. Pasamos el data a .JSON, para poder importarlo en MongoDB Compass:

In [23]:
df3.to_json("companies_with_coords.json", orient="records", lines=True)

15. Una vez que en MongoDB ya hemos creado un index geographical en nuestro .JSON, aplicamos la siguiente función para que nos indique, de cada una de las empresas que quedan en nuestra seleccion, cuantas empresas tienen cerca(entre 0 y 10km); el maximo es 25:

In [24]:
cursor4= db.companiesjson.find()
companies_found = []
for i in cursor4:
    cursor5 = db.companiesjson.find({
              "position": {
                "$near": {
                  "$geometry": {
                    "type": "Point",
                    "coordinates": [i['position']['coordinates'][0], i['position']['coordinates'][1]]
      },
      "$minDistance": 0,
      "$maxDistance":10000
    }
  }
})
    companies_found.append(cursor5.count())

print(max(companies_found))
    
#import pandas as pd
#df4 = pd.DataFrame(list(cursor4))
#df4.head()

/home/laura/.local/lib/python3.6/site-packages/ipykernel_launcher.py:16: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  app.launch_new_instance()


25


In [ ]:
16. Filtramos las empresas que mas empresas tienen alrededor, y nos queda una lista de 13:

In [19]:
cursor4= db.companiesjson.find()
companies_found = []
for i in cursor4:
    cursor5 = db.companiesjson.find({
              "position": {
                "$near": {
                  "$geometry": {
                    "type": "Point",
                    "coordinates": [i['position']['coordinates'][0], i['position']['coordinates'][1]]
      },
      "$minDistance": 0,
      "$maxDistance":10000
    }
  }
})
    if cursor5.count() == 25:
        companies_found.append(i)
    
import pandas as pd
df4 = pd.DataFrame(list(companies_found))
df4

/home/laura/.local/lib/python3.6/site-packages/ipykernel_launcher.py:16: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  app.launch_new_instance()


,_id,category_code,founded_year,name,number_of_employees,position,total_money_raised
0,5c6af01fadf5549f750d37a8,web,2008,BeliefNet,NaN,"{'type': 'Point', 'coordinates': [-73.985433, ...",$7M
1,5c6af01fadf5549f750d37af,web,2008,Kluster,NaN,"{'type': 'Point', 'coordinates': [-73.985281, ...",$1M
2,5c6af01fadf5549f750d37c5,web,2008,Bitly,NaN,"{'type': 'Point', 'coordinates': [-74.00721, 4...",$31.4M
3,5c6af02fadf5549f750d37f9,web,2010,Yipit,23.0,"{'type': 'Point', 'coordinates': [-73.987764, ...",$7.55M
4,5c6af02fadf5549f750d37fd,web,2008,Modelinia,NaN,"{'type': 'Point', 'coordinates': [-73.990024, ...",$5M
5,5c6af02fadf5549f750d381b,games_video,2008,Crispy Gamer,NaN,"{'type': 'Point', 'coordinates': [-73.985273, ...",$8.25M
6,5c6af02fadf5549f750d3823,ecommerce,2008,Proper Cloth,1.0,"{'type': 'Point', 'coordinates': [-74.008767, ...",$100k
7,5c6af02fadf5549f750d3827,software,2009,GameChanger Media,20.0,"{'type': 'Point', 'coordinates': [-74.0136605,...",$3.64M
8,5c6af02fadf5549f750d3828,ecommerce,2008,IntheGlo,5.0,"{'type': 'Point', 'coordinates': [-73.987764, ...",$30k
9,5c6af02fadf5549f750d3831,games_video,2008,Moogi,NaN,"{'type': 'Point', 'coordinates': [-74.0094471,...",$100k


17. Para poder situar estas empresas en un mapa, a través de Tableau, necesitamos una columna con las longitudes y otra con las latitudes:

In [20]:
df4.position[0]['coordinates'][0]   #el primer registro, la primera coordinates, (longitude)

df4['long'] = df4.apply(lambda row: row['position']['coordinates'][0], axis=1)
df4['lat'] = df4.apply(lambda row: row['position']['coordinates'][1], axis=1)

df4.head()

,_id,category_code,founded_year,name,number_of_employees,position,total_money_raised,long,lat
0,5c6af01fadf5549f750d37a8,web,2008,BeliefNet,NaN,"{'type': 'Point', 'coordinates': [-73.985433, ...",$7M,-73.985433,40.740231
1,5c6af01fadf5549f750d37af,web,2008,Kluster,NaN,"{'type': 'Point', 'coordinates': [-73.985281, ...",$1M,-73.985281,40.723652
2,5c6af01fadf5549f750d37c5,web,2008,Bitly,NaN,"{'type': 'Point', 'coordinates': [-74.00721, 4...",$31.4M,-74.007210,40.740340
3,5c6af02fadf5549f750d37f9,web,2010,Yipit,23.0,"{'type': 'Point', 'coordinates': [-73.987764, ...",$7.55M,-73.987764,40.744618
4,5c6af02fadf5549f750d37fd,web,2008,Modelinia,NaN,"{'type': 'Point', 'coordinates': [-73.990024, ...",$5M,-73.990024,40.739867


18. Nos quedamos solo con las columnas que nos interesa mostrar en Tableau:

In [21]:
df4.drop(['_id', 'position'], inplace=True, axis=1)

19. Guardamos nuestro resultado como .JSON: 

In [60]:
df4.to_json('visualize_companies', orient="records", lines=True)

20. Conectamos desde Tableau con nuestro .JSON, y vemos el mapa:

https://eu-west-1a.online.tableau.com/t/lausite/views/Companies_NYC20190219/Hoja1?iframeSizedToWindow=true&:embed=y&:showAppBanner=false&:display_count=no&:showVizHome=no

He elegido como nueva ubicacion las coordenadas (-73.99010, 40.74505), que corresponde a la direccion 34 W 27th St, New York, NY 10001, EE. UU, ya que actualmente alli esta establecida una empresa de 60 empleados, por lo que las instalaciones seran adecuadas para la nuestra de 50.

Con Google Maps tenemos mas informacion sobre la zona:

https://goo.gl/maps/4wkJFiuAc912

Y podemos ver fotos de la propiedad en el siguiente enlace:

https://streeteasy.com/building/43-west-27-street-new_york